PyTorch는 torch.nn 을 포함하여 신경망을 만들고 훈련시키는 것을 도울 수 있도록 섬세하게 만들어진 모듈과 클래스들을 제공한다.<br>
nn.Module 은 계층, 그리고 output을 반환하는 forward(input) 메소스들 포함하고 있다.

## 설치


In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


## 단계
1. 데이터를 가져오기 위해 필요한 라이브러리들 불러오기
2. 신경망을 정의하고 초기화
3. 데이터가 모델을 어떻게 지나갈 지 구체화
4. [선택사항] 데이터를 모델에 적용해 테스트하기

### 1. 데이터를 가져오기 위해 필요한 라이브러리들 불러오기
이 레시피에서, ```torch``` 와 이것의 하위 모듈인 ```torch.nn```, ```torch.nn.functional``` 을 사용

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### 2. 신경망을 정의하고 초기화
이미지를 인식하는 신경망을 만든다. PyTorch에서 만들어진 합성곱(convolution)이라고 불리는 방법을 사용한다. <br>
합성곱은 커널이나 작은 행렬(matrix)를 통해 가중치를 부여한 이미지의 각 요소를 주변 값과 더한다. 그리고 이것은 입력된 이미지의 특징(모서리 감지, 선명함, 흐릿함 등과 같은)을 추출하는 데 도움을 준다. <br><br>
모델의 ```Net``` 클래스를 정의하기 위해 2가지가 필요하다. 첫번재는 ```nn.Module```을 참고하는 ```__init__```함수를 작성하는 것이다. 이 함수는 신경망에서 fully connected layers를 만드는 것에 사용된다.<br><br>
합성곱을 사용해, 1개의 입력 이미지 채널을 가지고 우리의 target인 0부터 9까지 숫자를 대표하는 10개의 라벨과 대응되는 값을 출력하는 모델을 정의한다.<br>
기본적인 MNIST 알고리즘을 따르도록한다.

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # 첫번째 2D 합성곱 계층
        # 1개의 입력 채널(이미지)을 받아들이고, 사각 커널 사이즈가 3인 32개의 합성곱 특징들을 출력
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        
        # 두번째 2D 합성곱 계층
        # 32개의 입력 계층을 받아들이고, 사각 커널 사이즈가 3인 64개의 합성곱 특징을 출력
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        
        # 인접한 픽셀들은 입력 확률에 따라 모두 0 값을 가지거나 혹은 모두 유효한 값이 되도록 만든다
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        
        # 첫번째 fully connected layer
        self.fc1 = nn.Linear(9216, 128)
        # 10개의 라벨을 출력하는 두번째 fully connected layer
        self.fc2 = nn.Linear(128, 10)
        
my_nn = Net()
print(my_nn)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


### 3. 데이터가 모델을 어떻게 지나갈 지 구체화
PyTorch를 사용해 모델을 생성할 때, 계산 그래프(즉, 신경망)에 데이터를 지나가게 하는 ```forward```함수를 정의해야한다. 이것은 feed-forward 알고리즘을 나타낸다.<br><br>
```forward```함수에서 어떠한 Tensor 연산자도 사용 가능하다

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
    # x는 데이터를 나타낸다
    def forward(self, x):
        # 데이터가 conv1을 지나간다
        x = self.conv1(x)
        # x를 ReLU 활성함수(rectified-linear activation function)에 대입한다.
        x = F.relu(x)
        
        x = self.conv2(x)
        x = F.relu(x)
            
        # x에 대해서 max pooling을 실행
        x = F.max_pool2d(x, 2)
        # 데이터가 dropout1을 지나간다
        x = self.dropout1(x)
        # start_dim=1으로 x를 압축
        x = torch.flatten(x, 1)
        # 데이터가 fc1을 지나간다
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
            
        # x에 softmax를 적용한다
        output = F.log_softmax(x, dim=1)
        return output

### 4. [선택사항] 데이터를 모델에 적용해 테스트
원하는 출력값을 받을 수 있는 지 확인하기 위해, 무작위의 데이터를 모델에 통과시켜 시험해본다

In [13]:
# 임의의 28x28 이미지로 맞춰줍니다.
random_data = torch.rand((1, 1, 28, 28))

my_nn = Net()
result = my_nn(random_data)
print (result)

tensor([[-2.4658, -2.2315, -2.3385, -2.1790, -2.3988, -2.3443, -2.2250, -2.4127,
         -2.1699, -2.3078]], grad_fn=<LogSoftmaxBackward>)


/Users/hongjin-u/opt/anaconda3/envs/Pytorch/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459064158/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [16]:
result.shape

torch.Size([1, 10])